# Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import pytest
import time

from matplotlib import pyplot as plt

from datasets import Dataset
import evaluate

from transformers import AutoTokenizer 
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import Trainer
from transformers import TrainingArguments

from sklearn import metrics

from collections import Counter
from collections import defaultdict

from tqdm.auto import tqdm

import sklearn
from sklearn.metrics import classification_report

# Загружаем данные

In [2]:
def read_file(filename: str, encoding: str = 'utf-8') -> str:
    """
    Функция для чтения содержимого файла.
    
    :param filename: Имя файла или путь к файлу, который нужно прочитать.
    :param encoding: Кодировка файла (используем 'utf-8' для нашей задачи).
    :return: Строка с содержимым файла.
    """
    with open(filename, 'r', encoding=encoding) as file:
        return file.read()

In [3]:
train_txt = read_file('train.txt')
val_txt = read_file('dev.txt')
test_txt =  read_file('test.txt')

# Предобработка данных

In [4]:
def preprocess_text_to_dataframe(text: str) -> pd.DataFrame:
    '''
    Преобразование входного текста в DataFrame.
    '''
    # Разбиваем текст на строки
    lines = text.split('\n')
    
    # Списки для хранения предложений и меток
    all_sentences, all_tags = [], []
    
    # Временные списки для текущего предложения и меток
    current_sentence, current_tags = [], []
    
    for line in lines:
        if line:  # Если строка не пустая
            word, tag_value = line.split(' ')
            current_sentence.append(word)
            current_tags.append(tag_value)
        else:  # Если строка пустая, то это конец текущего предложения
            all_sentences.append(current_sentence)
            all_tags.append(current_tags)
            current_sentence, current_tags = [], []
    
    # Создаём DataFrame
    df = pd.DataFrame({'Words': all_sentences, 'Tags': all_tags})

    return df

In [5]:
train_data = preprocess_text_to_dataframe(train_txt)
val_data = preprocess_text_to_dataframe(val_txt)
test_data = preprocess_text_to_dataframe(test_txt)

In [6]:
# Объединение датафреймов в словарь
dataframes = {
    'train_data': train_data,
    'val_data': val_data,
    'test_data': test_data
}

# Вывод размерностей каждого датафрейма
dimensions = {key: df.shape for key, df in dataframes.items()}
dimensions

{'train_data': (7747, 2), 'val_data': (2583, 2), 'test_data': (2583, 2)}

In [7]:
train_data.head()

,Words,Tags
0,"["", Если, Миронов, занял, столь, оппозиционную...","[O, O, B-PER, O, O, O, O, O, O, O, O, O, O, O,..."
1,"[Источник, "", Ъ, '', в, руководстве, столичной...","[O, O, B-ORG, O, O, O, O, O, O, O, O, O, O, B-..."
2,"[В, Ханты-Мансийском, автономном, округе, с, д...","[O, B-LOC, I-LOC, I-LOC, O, O, O, O, B-ORG, B-..."
3,"[С, 1992, года, по, настоящее, время, является...","[O, O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, ..."
4,"[Для, этого, ей, пришлось, выиграть, выборы, в...","[O, O, O, O, O, O, O, O, O, O, B-LOC, I-LOC, O..."


In [8]:
def extract_unique_tags(df):
    """
    Извлекает уникальные теги из столбца 'Tags' датафрейма.

    Параметры:
    - df (pd.DataFrame): Датафрейм, содержащий столбец 'Tags' со списками тегов.
    """
    
    ner_tags = []  # Инициализация пустого списка для тегов

    # Проход по каждому списку тегов в датафрейме
    for tags_list in df['Tags']:
        # Добавление каждого тега в список ner_tags
        for tag in tags_list:
            ner_tags.append(tag)

    # Удаление дубликатов путем преобразования в множество и обратно в список
    ner_tags = list(set(ner_tags))

    return ner_tags

unique_tags = extract_unique_tags(train_data)
unique_tags

['I-LOC', 'I-ORG', 'I-PER', 'B-ORG', 'B-PER', 'B-LOC', 'O']

In [9]:
def create_label_mappings(df):
    """
    Извлекает уникальные теги из датафрейма и создает два словаря:
    tag_to_index (метка -> индекс) и index_to_tag (индекс -> метка).

    Параметры:
    - df (pd.DataFrame): Датафрейм, содержащий столбец 'Tags' со списками тегов.
    """
    
    # Извлечение уникальных тегов
    unique_tags = extract_unique_tags(df)
    
    # Создание словаря tag_to_index
    tag_to_index = dict(zip(unique_tags, range(len(unique_tags))))
    
    # Создание обратного словаря index_to_tag
    index_to_tag = {v: k for k, v in tag_to_index.items()}
    
    return tag_to_index, index_to_tag

tag_to_index, index_to_tag = create_label_mappings(train_data)
print(f'Словарь 1: {tag_to_index}')
print(f'Словарь 2: {index_to_tag}')

Словарь 1: {'I-LOC': 0, 'I-ORG': 1, 'I-PER': 2, 'B-ORG': 3, 'B-PER': 4, 'B-LOC': 5, 'O': 6}
Словарь 2: {0: 'I-LOC', 1: 'I-ORG', 2: 'I-PER', 3: 'B-ORG', 4: 'B-PER', 5: 'B-LOC', 6: 'O'}


In [10]:
def label_encode(labels):
    """
    Преобразует список меток в список соответствующих идентификаторов 
    с помощью словаря tag_to_index.

    Параметры:
    - labels (list): Список меток.

    Возвращает:
    - list: Список идентификаторов.
    """
    new_labels = [tag_to_index[label] for label in labels]
    return new_labels

# Применяем функцию к столбцу 'Tags' каждого датафрейма
train_data['Tags'] = train_data['Tags'].apply(label_encode)
val_data['Tags'] = val_data['Tags'].apply(label_encode)
test_data['Tags'] = test_data['Tags'].apply(label_encode)

train_data.head()

,Words,Tags
0,"["", Если, Миронов, занял, столь, оппозиционную...","[6, 6, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ..."
1,"[Источник, "", Ъ, '', в, руководстве, столичной...","[6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 6, ..."
2,"[В, Ханты-Мансийском, автономном, округе, с, д...","[6, 5, 0, 0, 6, 6, 6, 6, 3, 4, 2, 6]"
3,"[С, 1992, года, по, настоящее, время, является...","[6, 6, 6, 6, 6, 6, 6, 6, 3, 1, 1, 6, 1, 6, 6, ..."
4,"[Для, этого, ей, пришлось, выиграть, выборы, в...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 0, 6, 6]"


In [11]:
# Преобразование датафреймов в датасеты
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

# Fine-tuning предварительно обученной модели LaBSE от HuggingFace - архитектура BERT.

In [12]:
# Определение пути модели
model_path = "surdan/LaBSE_ner_nerel"

# Инициализация токенизатора
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [13]:
# Функция для выравнивания меток с токенами
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
            if label % 2 == 1:
                label += 1
            new_labels.append(label)
    return new_labels

In [14]:
# Функция для токенизации и выравнивания меток
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['Words'], truncation=True, is_split_into_words=True
    )
    all_labels = examples['Tags']
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
    tokenized_inputs['labels'] = new_labels
    return tokenized_inputs

In [15]:
# Применяем функцию к каждому датасету
tokenized_train = train_dataset.map(
    tokenize_and_align_labels, batched=True, remove_columns=train_dataset.column_names
)
tokenized_val = val_dataset.map(
    tokenize_and_align_labels, batched=True, remove_columns=val_dataset.column_names
)
tokenized_test = test_dataset.map(
    tokenize_and_align_labels, batched=True, remove_columns=test_dataset.column_names
)

Map:   0%|          | 0/7747 [00:00<?, ? examples/s]

Map:   0%|          | 0/2583 [00:00<?, ? examples/s]

Map:   0%|          | 0/2583 [00:00<?, ? examples/s]

In [16]:
# Инициализация инструмента для создания батчей
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Создаем словарь для преобразования меток
id2label = {i: label for i, label in enumerate(tag_to_index)}
label2id = {label: i for i, label in id2label.items()}

In [17]:
# загрузка метрик
metric = evaluate.load('seqeval')

In [18]:
# Функция для вычисления метрик
def compute_metric(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        'precision': all_metrics['overall_precision'],
        'recall'   : all_metrics['overall_recall'],
        'f1'       : all_metrics['overall_f1'],
        'accuracy' : all_metrics['overall_accuracy'],
    }

In [19]:
# Загрузка модели
model = AutoModelForTokenClassification.from_pretrained(
    model_path, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

# Параметры обучения
training_args = TrainingArguments(
    model_path,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at surdan/LaBSE_ner_nerel and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([58, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([58]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Инициализация тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    compute_metrics=compute_metric,
    tokenizer=tokenizer,
)

In [21]:
# Обучение модели
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.099000,0.024955,0.967339,0.979380,0.973322,0.993531
2,0.018300,0.024986,0.977580,0.980123,0.978850,0.994374
3,0.007200,0.025799,0.977346,0.981795,0.979565,0.994763


TrainOutput(global_step=2907, training_loss=0.03066817214081367, metrics={'train_runtime': 189.4263, 'train_samples_per_second': 122.691, 'train_steps_per_second': 15.346, 'total_flos': 720615539097696.0, 'train_loss': 0.03066817214081367, 'epoch': 3.0})

- `Training Loss и Validation Loss`: Модель успешно обучилась, так как потери на тренировочных данных уменьшались с каждой эпохой. Однако стоит обратить внимание, что потери на валидационных данных немного увеличились на второй эпохе, но затем снова уменьшились на третьей. Это может указывать на небольшое переобучение, но разница не критична.
- `F1`: Значение 0.97 говорит о высоком качестве нашей модели.

Можно сделать вывод, что модель показывает отличные результаты на валидационных данных.

In [22]:
# Предсказания на тестовом датасете
predictions = trainer.predict(tokenized_test)

In [23]:
# Извлечение истинных и предсказанных меток
labels_true = [l for s in tokenized_test['labels'] for l in s+[-100]*(predictions[0].shape[1] - len(s))]
labels_true = [0 if l==-100 else l for l in labels_true]
labels_pred = [np.argmax(l) for s in predictions[0] for l in s]

# Подсчет метрик
classification_report = metrics.classification_report(labels_true, labels_pred, labels=list(id2label.keys())[1:])

print(classification_report)

              precision    recall  f1-score   support

           1       0.92      0.96      0.94      1090
           2       0.98      0.99      0.98      4187
           3       0.96      0.98      0.97      1734
           4       0.97      0.99      0.98      7109
           5       0.99      0.98      0.98      1508
           6       0.43      1.00      0.60     63781

   micro avg       0.48      0.99      0.65     79409
   macro avg       0.88      0.98      0.91     79409
weighted avg       0.54      0.99      0.68     79409



- Классы 1, 2, 3, 4, 5: Эти классы показывают отличные результаты. Точность, полнота и F1-мера для этих классов находятся на уровне 96%-99%, что говорит о высокой эффективности нашей модели для этих классов.

- Класс 6: Этот класс имеет очень высокую полноту 100%, но относительно низкую точность 43%. Это означает, что наша модель почти всегда правильно идентифицирует истинные экземпляры этого класса (отсюда высокая полнота), но также часто ошибочно помечает другие классы как класс 6 (отсюда низкая точность). F1-мера для этого класса составляет 60%, что является средним значением между точностью и полнотой.

Среднее значение метрик для каждого класса (macro avg) F1-меры составляет 91% и это говорит о том, что в среднем по всем классам наша модель работает хорошо.
Среднее значение метрик для каждого класса (weighted avg), взвешенное по количеству экземпляров в каждом классе F1-меры в 68% указывает на хорошую производительность модели, учитывая распределение классов.

**Вывод**: Наша модель показывает отличные результаты для большинства классов. Однако для класса 5 производительность ниже, что может быть связано с несбалансированностью классов или другими факторами. Метрика F1  в среднем достигает 91% для всех классов, что является более чем положительным результатом в рамках домашнего задания.

**Общие выводы**: 

- 1. В коде явно не указано использование lr scheduler. Однако стоит отметить, что TrainingArguments в библиотеке transformers по умолчанию использует lr scheduler. 
- 2. В коде используется модель surdan/LaBSE_ner_nerel из библиотеки transformers, которая представляет собой одну из современных архитектур для NER. LaBSE (Language-agnostic BERT Sentence Embedding) — это одна из вариаций BERT, предназначенная для создания многоязычных эмбеддингов предложений.
- 3. В TrainingArguments указан параметр weight_decay, который является коэффициентом L2-регуляризации. Это помогает предотвратить переобучение модели.
- 4. Используется стандартная функция потерь для задачи классификации токенов в AutoModelForTokenClassification. Эта функция потерь обычно основана на кросс-энтропии.